In [3]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

import os
from dotenv import load_dotenv
load_dotenv() # load all env variables

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

from langchain_groq import ChatGroq
model = ChatGroq(model = "llama-3.1-8b-instant",groq_api_key=groq_api_key)

chain = prompt|model

In [4]:
from langchain_core.messages import HumanMessage

chain.invoke({"messages":[HumanMessage(content = "Hi, My name is Deepdas Rai.")]})

AIMessage(content="Nice to meet you, Deepdas Rai. I'm here to assist you with any questions or information you might need. What's on your mind today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 61, 'total_tokens': 93, 'completion_time': 0.063864751, 'prompt_time': 0.003696835, 'queue_time': 0.055488455, 'total_time': 0.067561586}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'finish_reason': 'stop', 'logprobs': None}, id='run--0c390488-9373-4df4-8be1-15c869ed22ab-0', usage_metadata={'input_tokens': 61, 'output_tokens': 32, 'total_tokens': 93})

In [5]:
# Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [7]:
config = {"configurable":{"session_id": "chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content = "Hi, My name is not Deepak Rai.")],
    config=config
)

response

AIMessage(content="It seems like you mentioned that earlier. You didn't provide your name, though. Feel free to share it if you'd like, or we can skip that part and move on to what's on your mind. What would you like to talk about?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 89, 'total_tokens': 141, 'completion_time': 0.122451556, 'prompt_time': 0.004886963, 'queue_time': 0.050065227, 'total_time': 0.127338519}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'finish_reason': 'stop', 'logprobs': None}, id='run--71f066ab-32d1-4b4b-b329-e28928b67d44-0', usage_metadata={'input_tokens': 89, 'output_tokens': 52, 'total_tokens': 141})

In [9]:
# Add more complexity to the prompt

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [10]:
response = chain.invoke(
    {
        "messages":[HumanMessage(content = "Hi, My name is Deepak Rai.")],
        "language": "Hindi"
     }
    )

response.content

'नमस्ते Deepak Rai जी, मुझे खुशी है आपकी मदद करने का मौका मिला। मैं आपके प्रश्नों का उत्तर देने के लिए तैयार हूँ।'

In [15]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [16]:
config = {"configurable":{"session_id": "chat4"}}

response = with_message_history.invoke(

    {
        "messages": [HumanMessage(content="Hi, My name is Deepak Rai")],
        "language":"Hindi"
     },
     config=config
)

response.content

'नमस्ते Deepak Rai, मैं आपकी मदद करने के लिए यहाँ हूँ। क्या मैं आपकी कोई समस्या का समाधान कर सकता हूँ या आपके लिए कुछ जानकारी प्रदान कर सकता हूँ?'

In [17]:
response = with_message_history.invoke(

    {
        "messages": [HumanMessage(content="Hi, What is my name")],
        "language":"Hindi"
     },
     config=config
)

response

AIMessage(content='आपका नाम Deepak Rai है।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 145, 'total_tokens': 156, 'completion_time': 0.013440589, 'prompt_time': 0.008036524, 'queue_time': 0.062877856, 'total_time': 0.021477113}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'finish_reason': 'stop', 'logprobs': None}, id='run--9c309524-c4fd-4d9e-8617-b90b73ad5360-0', usage_metadata={'input_tokens': 145, 'output_tokens': 11, 'total_tokens': 156})

In [18]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage,HumanMessage

trimmer = trim_messages(
    max_tokens=90,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi, I am Deepak. Do you remember me?"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like strawberry ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")

]

trimmer.invoke(messages)

d:\Data\Git Projects\Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi, I am Deepak. Do you remember me?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like strawberry ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [19]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
      "messages":messages + [HumanMessage(content="What ice cream do I like?")],
      "language":"English"
    }

)

response.content

'You like strawberry ice cream!'

In [20]:
response = chain.invoke(
    {
      "messages":messages + [HumanMessage(content="What math problem did I ask?")],
      "language":"English"
    }

)

response.content

'You asked the math problem 2 + 2.'

In [21]:
# let's wrap this in the form of message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config= {"configurable":{"session_id":"chat5"}}

In [22]:
response = with_message_history.invoke(

    {
        "messages":[HumanMessage(content="Hi, What is my name?")],
        "language":"English"
     },
     config=config
)

response.content

"I don't have any information about your name. Our conversation just started, and I'm a new assistant for each conversation. I don't have personal knowledge or memories of previous conversations. If you'd like to share your name, I'd be happy to know it."

In [23]:
response = with_message_history.invoke(

    {
        "messages":[HumanMessage(content="Hi, What math problem did I ask?")],
        "language":"English"
     },
     config=config
)

response.content

"You didn't ask a math problem in our conversation so far. Our conversation just started with your question about your name. If you'd like to ask a math problem or any other type of question, I'm here to help."